In [0]:
import numpy as np

In [0]:
import random
def create_u():
  """
  Inputs:
    None
  Outputs:
    u: +1 or -1 randomly generated
  """
  # YOUR CODE HERE
  u=[-1,1][random.randrange(2)]
  return u

In [18]:
new = create_u()
print(new)

-1


### Generate Data
Create a function that randomly gives either +1 or -1 as input

#### Desired data
$y(t+1) = w_0y(t) + w_1y(t-1) + w_2u(t) $

In [0]:
"""Test for create_u"""
u = create_u()
assert u==1 or u== -1


In [0]:
def generate_data(timesteps, w0, w1, w2):
  """
  Inputs:
    timesteps: int, number of timesteps of data
  Outputs:
    u: numpy array with each element +1 or -1 of shape (timesteps, )
    yd: numpy array with each element computed using above equation of shape (timesteps, )
  """
  # YOUR CODE HERE
  u =np.zeros(timesteps,)
  for i in range(timesteps):
    u[i]=create_u()
  yd=np.zeros(timesteps,)
  yd[0]=w2*u[0]
  yd[1]=w1*yd[0]+w2*u[0]
  for i in range(2,timesteps):
    yd[i]=w0*yd[i-1]+w1*yd[i-2]+w2*u[i-1]
  
  
  return u, yd


In [0]:
### Generate u, yd
u, yd = generate_data(10000, 1.1, -0.3, 0.5)

### Write expression for $y(t+1)$

In [0]:
def get_y_tplus1(w, yt, ytminus1, u):
  """
  Inputs:
    w: numpy array of weights of shape (3, )
    yt: float, y value at time t
    ytminus1: float, y value at time t - 1
    u: input at time t
  Outputs:
    y_tplus1: float, y at t+1
  """
  # YOUR CODE HERE
  y_tplus1=w[0]*yt+w[1]*ytminus1+w[2]*u
  return y_tplus1

### Forward propagation of weights
$$
\frac{\partial y(t+1)}{\partial w_0} = w_0\frac{\partial y(t)}{\partial w_0} + y(t) + w_1\frac{\partial y(t-1)}{\partial w_0}
$$
<br><br>
$$
\frac{\partial y(t+1)}{\partial w_1} = w_0\frac{\partial y(t)}{\partial w_1} + y(t-1) + w_1\frac{\partial y(t-1)}{\partial w_1} \\
$$
<br><br>
$$
\frac{\partial y(t+1)}{\partial w_2} = w_0\frac{\partial y(t)}{\partial w_2} +  w_1\frac{\partial y(t-1)}{\partial w_2} + u(t) 
$$
<br><br>
pwj1 is a matrix $\frac{\partial y(t)}{\partial w} =
[\frac{\partial y(t)}{\partial w_0}, \frac{\partial y(t)}{\partial w_1}, \frac{\partial y(t)}{\partial w_2}]$
<br><br>
pwj0 is a matrix $\frac{\partial y(t-1)}{\partial w} =
[\frac{\partial y(t-1)}{\partial w_0}, \frac{\partial y(t-1)}{\partial w_1}, \frac{\partial y(t-1)}{\partial w_2}]$
<br><br>
pwj2 is a matrix $\frac{\partial y(t+1)}{\partial w} =
[\frac{\partial y(t+1)}{\partial w_0}, \frac{\partial y(t+1)}{\partial w_1}, \frac{\partial y(t+1)}{\partial w_2}]$

In [0]:
def forward_prop_weights(w, yt, ytminus1, u, pwj1, pwj0):
  """
  Inputs:
    w: numpy array of weights [w0, w1, w2]
    yt: float, y value at time t
    ytminus1: float, y value at time t - 1
    u: input at time t
    pwj1: numpy array of shape (3, ) as described above
    pwj0: numpy array of shape (3, ) as described above
  Outputs:
    pwj2: numpy array of shape (3, ) as described above
  """  
  # YOUR CODE HERE
  pwj2=np.zeros(3,)
  pwj2[0]=w[0]*pwj1[0]+yt+w[1]*pwj0[0]
  pwj2[1]=w[0]*pwj1[1]+ytminus1+w[1]*pwj0[1]
  pwj2[2]=w[0]*pwj1[2]+w[1]*pwj0[2]+u
  
  return pwj2

### Weight update - Gradient descent
$E = \frac{1}{2}(y_d - y)^2$ <br>
$w_{new} = w_{old} - \eta\frac{\partial E}{\partial w} = w_{old} - \eta(y_d - y)\cdot pwj2$ 

In [0]:
def update_w(w, eta, yd, y, pwj2):
  """
  Inputs:
    w: numpy array of weights [w0, w1, w2]
    eta: float, learning rate
    yd: scalar(float), desired value of y at time t+1
    y: scalar(float), predicted value of y at time t+1
    pwj2: numpy array of shape(3,) obtained from forward_prop_weights
  Outputs:
    w_updated: numpy array of shape(3, ), updated weights
  """
  # YOUR CODE HERE
  w_updated=[]
  for i in range(len(w)):
    w_updated.append(w[i]-eta*(yd-y)*pwj2[i])
  w_updated=np.array(w_updated)
  
  return w_updated

### Cost function (error)
$E = \frac{1}{2}(y_d - y)^2$ <br>


In [0]:
def err(yd, y):
  """
  Inputs:
    yd: scalar(float), desired value of y at time t+1
    y: scalar(float), predicted value of y at time t+1
  Outputs:
    E: scalar(float), error as described above 
  """
  # YOUR CODE HERE
  E = 0.5*(yd-y)**2
  return E

### Apply RTRL

In [0]:
def RTRL(yd, eta, timesteps):
  """
  Inputs:
    yd: numpy array of shape(timesteps,) 
    eta: scalar(float), learning rate
    timesteps: int, number of timesteps
  Outputs:
    west: numpy array of shape(timesteps,3), keeps track of all the updated weights over time
    error: numpy array of shape(timesteps,), keeps track of error (as obtained from function err()) over time
  """
  # YOUR CODE HERE
  u=create_u()
  w=np.zeros(3,)
  y=np.random.random(timesteps,)
  yt=0
  ytminus1=0
  pwj1=np.zeros(3,)
  pwj0=np.zeros(3,)
  west=[]
  error=[]
  
  for k in range(timesteps):
    pwj2=forward_prop_weights(w, yt, ytminus1, u, pwj1, pwj0)
    w_updated = update_w(w, eta, yd, y, pwj2)
    E = err(yd, y)
    west.append(w_updated)
    error.append(E)
  west=np.array(west)
  error=np.array(error)
  return west, error
   
   

In [0]:
def plot_w(west):
  plt.plot(west[:,0])
  plt.show()
  plt.plot(west[:,1])
  plt.show()
  plt.plot(west[:,2])
  plt.show()

In [0]:
def plot_err(error):
  plt.plot(error)
  plt.show()

In [15]:
import numpy as np
import matplotlib.pyplot as plt

eta = 0.005
iterations = 10000

west, error = RTRL(yd, eta, iterations)

plot_w(west)
plot_err(error)

NameError: ignored

In [16]:
w = np.array([1.1, -0.3, 0.5])
print(w[0])

1.1
